In [23]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import nltk
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [24]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/happy_moss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Я буду разделять книги написанные Виктором Пелевиным и Антоном Чеховым используя метод сравнения корпуса текстов Килгриффа. В кратце он заключается в этом: мы объединяем корпус слов возможного автора с некласифицированным корпусом, вычисляем проецированный вклад и реальный вклад автора в корпус, а далее хи-квадрат между этими значениями и суммируем се для каждого слова. Автор с наименьшим отклонением является наиболее вероятным.

_Публикация описывающая алгоритм: https://www.kilgarriff.co.uk/Publications/2001-K-CompCorpIJCL.pdf_

Для начала загрузим датасет

In [25]:
p = Path("./")
book_paths = p.glob("**/*.txt")
books = []
for bp in book_paths:
    books.append((bp.read_text(encoding="utf-8-sig"), "Pelevin" if bp.parent == Path("./books/Pelevin") else "Chekhov"))

In [26]:
books_df = pd.DataFrame(columns=["book_text", "author"], data=books)

Можно взять часть датасета, но я использую новые данные для тестов: полный текста "Архиерея", первую главу "Transhumanism Inc.", последнее действие "Вишневого Сада", третью главу "Тайные виды на гору Фудзи"

In [27]:
from distinguish_test_data import test_data
test_data_df = pd.DataFrame(columns=["text", "author"], data=test_data)

In [28]:
X_train = books_df["book_text"]
y_train = books_df["author"]
X_test = test_data_df["text"]
y_test = test_data_df["author"]

In [29]:
def text_tokenize(t):
    
    stopwords_ru = stopwords.words("russian")
    stopwords_ru.extend(["это"])

    tokenized_text = [
        w for w in word_tokenize(t.lower())
        if ((not w in stopwords_ru) and (w.isalpha()))
    ]

    return tokenized_text

Класс который имплементирует алгоритм описанный в статье

In [30]:
class KilgariffCC:
    def __init__(self, X, y):
        assert len(X) == len(y)

        self.tok_dicts = {}
        for t in zip(X, y):
            if self.tok_dicts.get(t[1]) is None:
                self.tok_dicts[t[1]] = []
            
            if len(self.tok_dicts[t[1]]) >= 150000:
                continue

            self.tok_dicts[t[1]].extend(text_tokenize(t[0]))



    def predict(self, uncategorized_texts):

        predictions = []

        for uncategorized_text in uncategorized_texts:
            uncategorized_tok = text_tokenize(uncategorized_text)

            authors_chi = {}
            for author in self.tok_dicts.keys():
                joint_tokens = uncategorized_tok + self.tok_dicts[author]

                joint_freq_dist = FreqDist(joint_tokens)
                joint_most_common_tok = joint_freq_dist.most_common(1000)
                
                author_share = (len(self.tok_dicts[author]) / len(joint_tokens))
                
                chisquared = 0
                for word,joint_count in joint_most_common_tok:

                    author_count = self.tok_dicts[author].count(word)
                    disputed_count = uncategorized_tok.count(word)

                    expected_author_count = joint_count * author_share
                    expected_disputed_count = joint_count * (1-author_share)

                    chisquared += ((author_count-expected_author_count) *
                                (author_count-expected_author_count) /
                                expected_author_count)

                    chisquared += ((disputed_count-expected_disputed_count) *
                                (disputed_count-expected_disputed_count)
                                / expected_disputed_count)
                    
                authors_chi[author] = chisquared
            
            lowest_chi = list(authors_chi.values())[0]
            probable_author = 0
            for author, chi in authors_chi.items():
                if chi <= lowest_chi:
                    lowest_chi = chi
                    probable_author = author

            predictions.append(probable_author)

        return predictions

In [ ]:
kcc = KilgariffCC(X_train, y_train)

In [ ]:
pred = kcc.predict(X_test)

В данном случае точность 100%. Это хороший определения авторства, однако его точность падает с уменьшением длинны неопределённого текста или если корпус одного из авторов значительно больше других. Проблему №2  мы митигируем используя примерно равное количество токенов обоих авторов в данном случае, однко пробема №1 нам не подвалстна.

In [ ]:
accuracy_score(y_true=y_test, y_pred=pred)

Если взять кластеры по 613 предложений из датасета и запускать предсакзания на них, то точность составит уже около 90% и так далее она будет снижаться по мере уменьшения входного корпуса.

In [ ]:
book_sents = []
for b in books:
    n = 0
    sent_cluster = ""
    for s in sent_tokenize(b[0], language="russian"):
        if n != 613 and len(s) != 0:
            sent_cluster = sent_cluster + s
            n += 1
        else:
            book_sents.append((sent_cluster, b[1]))
            sent_cluster = ""
            n = 0

In [ ]:
book_sents_df = pd.DataFrame(columns=["sentances", "author"], data=book_sents)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    book_sents_df["sentances"],
    book_sents_df["author"],
    test_size=0.2,
    random_state=587)

In [ ]:
kcc = KilgariffCC(X_train, y_train)

In [ ]:
pred = kcc.predict(X_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=pred)